In [1]:
# list datalab file
!ls /home/anhaoran/data/zero-shot-tianchi/dataset_B/DatasetB_20180919/

attribute_list.txt	  class_wordembeddings.txt  label_list.txt  train
attributes_per_class.txt  image.txt		    test	    train.txt


之江这道题其实格外提供了一个属性表，包括类别、颜色、用途等；其实是降低了难度，那我们可以简单点先做一版。
这里baseline仅使用这31维的属性标签做训练，最后对比这31维的向量，用np.linalg.norm计算欧式距离，然后从40个新类别中选取最接近的lable。
分析这31维的属性其实是分为5个大类若干小类 所以我们用联合训练的方法训练5个分类器出来

In [2]:
import numpy as np
 
f = open(r'/home/anhaoran/data/zero-shot-tianchi/dataset_B/DatasetB_20180919//attributes_per_class.txt')
data_per_class = f.readlines()
 
data = []
for line in data_per_class:
    data.append(line.split())
print(data)
print(np.shape(data))

[['ZJL1', '1', '0', '0', '0', '0', '0', '0.5', '1', '0', '0', '1', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['ZJL10', '1', '0', '0', '0', '0', '0', '0.5', '0', '0', '0.7', '0', '0', '0', '0.2', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['ZJL100', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0'], ['ZJL101', '0', '0', '0', '0', '0', '0', '0.8', '0.1', '0.3', '0.1', '0.3', '0.1', '0.3', '0.3', '0', '0', '0', '0', '1', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0'], ['ZJL102', '0', '0', '0', '0', '0', '0', '0', '0.1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0.5', '0', '0', '0', '0', '0', '0', '0'], ['ZJL103', '0', '0', '0', '0', '0', '0', '0', '0.5', '0.5', '0.5', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0

In [3]:
#这里的属性标签其实分为类别、颜色、用途等5类，
#但是最后一个类别主观太重，估计打标的同学也很为难，我就不用了，还有颜色这个我暂且不用，比如car\pan\cost 这些颜色有点误导吧
head = list(['lable',
    'animal',
'transportation',
'clothes',
'plant',
'tableware',
'device',
'black',
'white',
'blue',
'brown',
'orange',
'red',
'green',
'yellow',
'has_feathers',
'has_four_legs',
'has_two_legs',
'has_two_arms',
'for_entertainment',
'for_business',
'for_communication',
'for_family',
'for_office use',
'for_personal',
'gorgeous',
'simple',
'elegant',
'cute',
'pure',
'naive'])

In [4]:
#对lable做一次清洗
import pandas as pd
data = pd.DataFrame(data,columns= head)
# data = data.set_index('lable')
 
data_1 = data.loc[:,'animal':'device']
data_2 = data.loc[:,'black':'yellow']
data_3 = data.loc[:,'has_feathers':'has_two_arms']
data_4 = data.loc[:,'for_entertainment':'for_personal']
 
data_1 = (data_1 >= '1') & 1
data_1['non_1'] = 1 - data_1.sum(axis = 1)
#print(data_1.shape)
 
#颜色属性不太好用（比如car\pan\cost），这里就不做复杂处理，模型训练时暂不用
data_2 = (data_2 > '0.7') & 1
data_2.sum(axis = 1)
#print(data_2)
 
data_3 = (data_3 >= '1') & 1
data_3.sum(axis = 1)
 
data_4 = (data_4 >= '1') & 1
data_4.sum(axis = 1)
 
data_new = pd.DataFrame(data['lable'])
data_new = pd.DataFrame(np.hstack((data_new, data_1, data_2, data_3, data_4)))
data_new.to_csv("test.csv", index=None)
print("data saved to test.csv.")

data saved to test.csv.


In [1]:
#build model
from keras import Model, Sequential
from keras.applications import VGG16, VGG19
from keras.layers import Flatten, Dense
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
 
base_model = VGG19(include_top=False, weights=None, input_shape=(64, 64, 3))
 
classes = {
    'cla' : 6+1,
  #  'clo' : 8, #暂且不用
    'has' : 4,
    'for' : 6
  #  'is' : 6 #暂且不用
}
 
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
predictions = [Dense(n, activation='softmax', name=m)(x) for m,n in classes.items()]
 
model = Model(inputs=base_model.input, outputs= predictions)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

/home/anhaoran/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 64, 64, 64)   1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 64, 64, 64)   36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 32, 32, 64)   0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [2]:
#这个数据集我只传了2张图片 所以你线下跑是注意下面标注的修改的地方
import pandas as pd
import numpy as np

data_atten = pd.read_csv(r'test_1.csv')
data_atten = data_atten.set_index('0')
data_train = open(r'/home/anhaoran/data/zero-shot-tianchi/dataset_A/train/train.txt')
data_train = data_train.readlines()[:2]#!!!!!!!!!!!!!!!!!!!!!！！！！！！！！！！！！！！！！！！这里要注意修改
print(data_train[0])
path = r'/home/anhaoran/data/zero-shot-tianchi/dataset_A/train/train/'
length = len(data_train)
train_x = np.zeros((length, 64, 64, 3))
train_y = np.zeros((length, 25))
for i in range(length):
    m,n = data_train[i].split()
    img = image.load_img(path + m)
    train_x[i] = image.img_to_array(img)
    train_y[i] = data_atten.loc[n]
    
#np.save('abc.npy', train_x)
#train_x = np.load('abc.npy')
h = model.fit(train_x, {'cla':train_y[:,:7], 'has':train_y[:,15:19], 'for':train_y[:,19:25]},epochs=50,validation_split=0.2)

a6394b0f513290f4651cc46792e5ac86.jpeg	ZJL1

Train on 1 samples, validate on 1 samples
Epoch 1/50
1/1 [==============================] - 11s 11s/step - loss: 3.8113 - cla_loss: 1.9901 - has_loss: 0.0000e+00 - for_loss: 1.8212 - val_loss: 2.3842e-07 - val_cla_loss: 1.1921e-07 - val_has_loss: 0.0000e+00 - val_for_loss: 1.1921e-07
Epoch 2/50
1/1 [==============================] - 0s 90ms/step - loss: 2.3842e-07 - cla_loss: 1.1921e-07 - has_loss: 0.0000e+00 - for_loss: 1.1921e-07 - val_loss: 2.3842e-07 - val_cla_loss: 1.1921e-07 - val_has_loss: 0.0000e+00 - val_for_loss: 1.1921e-07
Epoch 3/50
1/1 [==============================] - 0s 85ms/step - loss: 2.3842e-07 - cla_loss: 1.1921e-07 - has_loss: 0.0000e+00 - for_loss: 1.1921e-07 - val_loss: 2.3842e-07 - val_cla_loss: 1.1921e-07 - val_has_loss: 0.0000e+00 - val_for_loss: 1.1921e-07
Epoch 4/50
1/1 [==============================] - 0s 89ms/step - loss: 2.3842e-07 - cla_loss: 1.1921e-07 - has_loss: 0.0000e+00 - for_loss: 1.1921e-07 - val_loss

Epoch 33/50
1/1 [==============================] - 0s 86ms/step - loss: 2.3842e-07 - cla_loss: 1.1921e-07 - has_loss: 0.0000e+00 - for_loss: 1.1921e-07 - val_loss: 2.3842e-07 - val_cla_loss: 1.1921e-07 - val_has_loss: 0.0000e+00 - val_for_loss: 1.1921e-07
Epoch 34/50
1/1 [==============================] - 0s 84ms/step - loss: 2.3842e-07 - cla_loss: 1.1921e-07 - has_loss: 0.0000e+00 - for_loss: 1.1921e-07 - val_loss: 2.3842e-07 - val_cla_loss: 1.1921e-07 - val_has_loss: 0.0000e+00 - val_for_loss: 1.1921e-07
Epoch 35/50
1/1 [==============================] - 0s 92ms/step - loss: 2.3842e-07 - cla_loss: 1.1921e-07 - has_loss: 0.0000e+00 - for_loss: 1.1921e-07 - val_loss: 2.3842e-07 - val_cla_loss: 1.1921e-07 - val_has_loss: 0.0000e+00 - val_for_loss: 1.1921e-07
Epoch 36/50
1/1 [==============================] - 0s 99ms/step - loss: 2.3842e-07 - cla_loss: 1.1921e-07 - has_loss: 0.0000e+00 - for_loss: 1.1921e-07 - val_loss: 2.3842e-07 - val_cla_loss: 1.1921e-07 - val_has_loss: 0.0000e+00 - v

In [3]:
model.load_weights("./vgg19/baseline_model.h5")

# Predicting

In [18]:
#下面的代码【预测及计算两两向量的欧氏距离】涉及的测试集没上传 注意不能直接运行，线下修改运行
 
data_test = open("/home/anhaoran/data/zero-shot-tianchi/dataset_A/test/image.txt")
data_test = data_test.readlines()
path_test = r'/home/anhaoran/data/zero-shot-tianchi/dataset_A/test/test/'
length = len(data_test)
data_test_x = np.zeros((length, 64, 64, 3))
for i in range(length):
    m = data_test[i][:-1]
    img = image.load_img(path_test + m)
    data_test_x[i] = image.img_to_array(img)
    
res = model.predict(data_test_x)

In [19]:
print(data_test_x.shape)
print(res[0].shape)
print(res[1].shape)
print(res[2].shape)

(14633, 64, 64, 3)
(14633, 7)
(14633, 4)
(14633, 6)


In [36]:
#欧氏距离：
print(np.shape(data_atten))
#print(data_atten.head)
data_atten_res = []
for i in range(160,241):
    key = 'ZJL' + str(i)
    data_atten_res.append(np.hstack((data_atten.loc[key][:7], data_atten.loc[key][15:19], data_atten.loc[key][19:])))
print(np.shape(data_atten_res))

vec_result = []#np.zores((len(res[0]), 18))
for i in range(len(res[0])):
    vec_result.append(np.hstack((res[0][i], res[1][i], res[2][i])))
print(np.shape(vec_result))

dis_list = [] 
for j in range(len(data_atten_res)):
    vec1 = vec_result[0]
    vec2 = data_atten_res[j]
    dist = np.linalg.norm(vec1 - vec2)
    dis_list.append(dist)
index = dis_list.index(min(dis_list))
print("witch = "+ str(index) + " and min dis = " + str(min(dis_list)) + " -> the label is ZJL" + str(index + 160))
print(data_atten_res[43])
print(np.hstack((data_atten.loc['ZJL203'][:7], data_atten.loc['ZJL203'][15:19], data_atten.loc['ZJL203'][19:])))
print(np.hstack((data_atten.loc['ZJL202'][:7], data_atten.loc['ZJL202'][15:19], data_atten.loc['ZJL202'][19:])))
print(np.hstack((data_atten.loc['ZJL204'][:7], data_atten.loc['ZJL204'][15:19], data_atten.loc['ZJL204'][19:])))
print(np.hstack((data_atten.loc['ZJL205'][:7], data_atten.loc['ZJL205'][15:19], data_atten.loc['ZJL205'][19:])))

(230, 25)
(81, 17)
(14633, 17)
witch = 43 and min dis = 1.0 -> the label is ZJL203
[1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


In [43]:
data_atten = pd.read_csv(r'test.csv')
data_atten = data_atten.set_index('0')
print(data_atten)

        1  2  3  4  5  6    7  8  9  10 ...  16  17  18  19  20  21  22  23  \
0                                       ...                                   
ZJL1    1  0  0  0  0  0  0.0  0  1   0 ...   0   0   0   0   1   0   0   0   
ZJL10   1  0  0  0  0  0  0.0  0  0   0 ...   0   0   0   0   0   0   0   0   
ZJL100  0  0  0  0  0  0  1.0  0  0   0 ...   0   0   0   0   1   0   0   0   
ZJL101  0  0  0  0  0  0  1.0  1  0   0 ...   0   0   0   0   1   0   0   1   
ZJL102  0  0  0  0  0  0  1.0  0  0   0 ...   0   0   0   0   0   0   0   0   
ZJL103  0  0  0  0  0  0  1.0  0  0   0 ...   0   0   0   0   1   1   0   1   
ZJL104  0  1  0  0  0  0  0.0  0  0   0 ...   0   0   0   0   0   1   0   1   
ZJL105  0  0  0  0  0  1  0.0  1  0   0 ...   0   0   0   0   0   1   0   1   
ZJL106  0  0  1  0  0  0  0.0  0  0   0 ...   0   0   0   0   0   1   0   0   
ZJL107  0  0  0  0  0  1  0.0  0  0   0 ...   0   0   0   0   1   1   0   0   
ZJL108  0  1  0  0  0  0  0.0  0  0   0 ...   0   0 

In [44]:
import numpy as np
 
f = open(r'/home/anhaoran/data/zero-shot-tianchi/dataset_A/train/attributes_per_class.txt')
data_per_class = f.readlines()
 
data = []
for line in data_per_class:
    data.append(line.split())
data = np.asarray(data)
data_attributes = np.asarray(data[:, 1:26], dtype=np.float32)
print(data_attributes)
print(data_attributes.shape)

[[1.  0.  0.  ... 0.  0.  0. ]
 [1.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  1.  0. ]
 ...
 [0.  1.  0.  ... 0.  0.  0. ]
 [0.  1.  0.  ... 0.  0.5 0. ]
 [1.  0.  0.  ... 0.  0.5 0. ]]
(230, 25)


In [45]:
attributes_dict = {}
index = 0
for item in data:
    if len(item[0]) < 6 or item[0] < 'ZJL201':
        continue
    attributes_dict.update({item[0] : data_attributes[index]})
    index += 1
print(attributes_dict)
print(len(attributes_dict))

{'ZJL201': array([1. , 0. , 0. , 0. , 0. , 0. , 0.5, 1. , 0. , 0. , 1. , 0. , 0. ,
       1. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. ],
      dtype=float32), 'ZJL202': array([1. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0.7, 0. , 0. , 0. ,
       0.2, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
      dtype=float32), 'ZJL203': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 1., 0.], dtype=float32), 'ZJL204': array([0. , 0. , 0. , 0. , 0. , 0. , 0.8, 0.1, 0.3, 0.1, 0.3, 0.1, 0.3,
       0.3, 0. , 0. , 0. , 0. , 1. , 0. , 0. , 1. , 0. , 1. , 0. ],
      dtype=float32), 'ZJL205': array([0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.1, 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. ],
      dtype=float32), 'ZJL206': array([0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0.5, 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 1. , 1. , 0. , 1. , 1. , 1. , 0. ],
      dtype=float32)

In [46]:
from keras.preprocessing import image

model_dir = "./inceptionv3/"
image_size = 139

data_test = open("/home/anhaoran/data/zero-shot-tianchi/dataset_A/test/image.txt")
data_test = data_test.readlines()
path_test = r'/home/anhaoran/data/zero-shot-tianchi/dataset_A/test/test/'
length = len(data_test)
data_test_x = np.zeros((length, image_size, image_size, 3))
for i in range(length):
    m = data_test[i][:-1]
    img = image.load_img(path_test + m, target_size=(image_size, image_size, 3))
    data_test_x[i] = image.img_to_array(img)

print(data_test_x.shape)

(14633, 139, 139, 3)


In [48]:
from keras.models import load_model

model = load_model(model_dir + 'inceptionv3_2018-08-24_baseline_model.h5')
res = model.predict(data_test_x)
    
vec_result = []
for i in range(len(res[0])):
    vec_result.append(np.hstack((res[j][i] for j in range(25))))

In [51]:
print(np.shape(vec_result))

(14633, 25)


In [55]:
submission = open('./submission-1.txt', 'w')

#for i in range(1):
for i in range(len(vec_result)):
    print(i, end=" : ")
    dis_list = []
    for j in range(len(attributes_dict)):
        vec1 = vec_result[i]
        vec2 = attributes_dict['ZJL'+str(j+201)]
        dist = np.linalg.norm(vec1 - vec2)
        dis_list.append(dist)
    index = dis_list.index(min(dis_list))
    print("witch = "+ str(index) + " and min dis = " + str(min(dis_list)) + " -> " + 
          data_test[i][:-1] + " label is ZJL" + str(index + 201))
    submission.write(data_test[i][:-1] + '\t' + "ZJL" + str(index+201) + '\n')
submission.close()

0 : witch = 4 and min dis = 1.3091114 -> 00ddbe75d7aff5037d360401af02ca57.jpg label is ZJL205
1 : witch = 1 and min dis = 1.1526299 -> 03357448dd6ce1ac0b5b69e9204c53ee.jpg label is ZJL202
2 : witch = 1 and min dis = 1.5956146 -> 051b37cb697a5dbd6230fa9b58bc170d.jpg label is ZJL202
3 : witch = 4 and min dis = 1.0683378 -> 08247ec84ef66743ac30ba51cf2c1607.jpg label is ZJL205
4 : witch = 4 and min dis = 1.3741174 -> 093e28418698ca07ca805377e4533f96.jpg label is ZJL205
5 : witch = 1 and min dis = 1.2508373 -> 0dc47ee8e5804b2fc35e453c7a5fec2d.jpg label is ZJL202
6 : witch = 4 and min dis = 1.1702551 -> 145b317ddf3abf424eec3aeded7e8af4.jpg label is ZJL205
7 : witch = 4 and min dis = 1.1475892 -> 14ffb0bdf864e72dddef53ae1e8933be.jpg label is ZJL205
8 : witch = 4 and min dis = 1.1046695 -> 156301bb40a3b8034aca3443a3a5a345.jpg label is ZJL205
9 : witch = 4 and min dis = 1.4796597 -> 183fb67145085b6f490362651225e539.jpg label is ZJL205
10 : witch = 4 and min dis = 1.5579729 -> 18c946fa057cf2ffac

231 : witch = 37 and min dis = 1.4069656 -> 674915a3a9302605d72bf5c1472a37d8.jpg label is ZJL238
232 : witch = 4 and min dis = 1.2984546 -> 69263ad7332d6b7a07e2ed6c9f449426.jpg label is ZJL205
233 : witch = 33 and min dis = 1.5079194 -> 695a63a4b113fdcc49dcd07b842e2b07.jpg label is ZJL234
234 : witch = 4 and min dis = 1.3182164 -> 6976328e95f5f3cabb12a227b00dd27a.jpg label is ZJL205
235 : witch = 1 and min dis = 1.4649323 -> 699fccaf1ac58a306816aac36bfd9771.jpg label is ZJL202
236 : witch = 1 and min dis = 1.2985805 -> 69f6f27eb5c315ddae2734a9a88ae2d9.jpg label is ZJL202
237 : witch = 4 and min dis = 1.0904181 -> 69fc54929af60289e9b51550363b69b1.jpg label is ZJL205
238 : witch = 1 and min dis = 1.0810854 -> 6ae7e6112383cdc9f9ab7ca0d6a5c1f4.jpg label is ZJL202
239 : witch = 33 and min dis = 1.6717942 -> 6b0b48e219a66ed7fcf2eec1ff8b0c3a.jpg label is ZJL234
240 : witch = 4 and min dis = 0.7269172 -> 6b9ec46bf932e80ba58050d6ae36cf6a.jpg label is ZJL205
241 : witch = 33 and min dis = 1.6854

469 : witch = 4 and min dis = 1.1224966 -> 0781b5950cfcdff0431f536ada490377.jpg label is ZJL205
470 : witch = 4 and min dis = 1.0201203 -> 07a195eab7ccad3a359664f20b9d1ffa.jpg label is ZJL205
471 : witch = 4 and min dis = 1.1868713 -> 0927e0b800767e9e5e94a888d87f7412.jpg label is ZJL205
472 : witch = 4 and min dis = 0.76906896 -> 092dbc769b0be8f966b386081ee2b39c.jpg label is ZJL205
473 : witch = 4 and min dis = 1.5941545 -> 094b0ed895180809bda56a55814f40c6.jpg label is ZJL205
474 : witch = 4 and min dis = 0.9918994 -> 0989b6b1b36d7bd048dd3d0e109422fe.jpg label is ZJL205
475 : witch = 4 and min dis = 1.0514648 -> 0a68f0e1d9f3eb6cc8042268667c5a75.jpg label is ZJL205
476 : witch = 4 and min dis = 1.4987246 -> 0a96354e25ae074be5607c749fd2296b.jpg label is ZJL205
477 : witch = 4 and min dis = 1.1746668 -> 0ac9ff183f3c1a902b594f96877d3e67.jpg label is ZJL205
478 : witch = 4 and min dis = 1.1662438 -> 0e4b5bcfe0f6d8ae6e32d2390d549fdc.jpg label is ZJL205
479 : witch = 4 and min dis = 1.150211 

661 : witch = 4 and min dis = 1.3336394 -> 9afdb60797de72f06bc1575ee3061370.jpg label is ZJL205
662 : witch = 4 and min dis = 1.7216102 -> 9c5ee0fe557aad4459053015d29bbe86.jpg label is ZJL205
663 : witch = 4 and min dis = 0.9891293 -> 9cd239ff4227ad3f513132b930544cc9.jpg label is ZJL205
664 : witch = 4 and min dis = 1.2970039 -> 9d537111ae5107e3a20e4b3a481fc988.jpg label is ZJL205
665 : witch = 4 and min dis = 1.2820941 -> 9dee162df48c1c2d261cd862cceb3758.jpg label is ZJL205
666 : witch = 4 and min dis = 1.1000011 -> 9e2c390ff99a214411d18410f8bb2c72.jpg label is ZJL205
667 : witch = 4 and min dis = 1.1629231 -> 9e89bc8aa975e1a903c52ba23a99826a.jpg label is ZJL205
668 : witch = 4 and min dis = 1.1036487 -> 9f1d5d8527cad7a30749d92957fc7d7b.jpg label is ZJL205
669 : witch = 4 and min dis = 1.644269 -> a01f645ecee10005f6b23128fecd397a.jpg label is ZJL205
670 : witch = 4 and min dis = 1.2592905 -> a02d0c03df88dc0daf2ef920f46f8ced.jpg label is ZJL205
671 : witch = 4 and min dis = 1.2039838 -

929 : witch = 4 and min dis = 1.4706901 -> 70545adac2903f66b9f31b4918e4da6b.jpg label is ZJL205
930 : witch = 4 and min dis = 1.2748303 -> 97e7f6e34a275258b1e29d1ac82dad6d.jpg label is ZJL205
931 : witch = 20 and min dis = 1.5733547 -> ad714a041cdc92b8f429bcea6e6f157e.jpg label is ZJL221
932 : witch = 4 and min dis = 1.2645026 -> ba55091d5654d1413b80f59dfbe2d4da.jpg label is ZJL205
933 : witch = 4 and min dis = 1.2565036 -> 27fca6bc27ddba3b29d01a4ef0d41240.jpg label is ZJL205
934 : witch = 20 and min dis = 1.4861223 -> a3f8779f2a5ffaa29ff4f50fe4cf62eb.jpg label is ZJL221
935 : witch = 31 and min dis = 1.7892153 -> 2bf0b48c49fd31600b1a863af4f28238.jpg label is ZJL232
936 : witch = 32 and min dis = 1.6731739 -> 4988a7d0b8cf69572c17b3da8443911a.jpg label is ZJL233
937 : witch = 4 and min dis = 1.3582116 -> 6cd1673f210237bfa4d1beaeb687e3db.jpg label is ZJL205
938 : witch = 4 and min dis = 1.491201 -> a7fefb60dfa299160367caf09c28dc40.jpg label is ZJL205
939 : witch = 4 and min dis = 1.19896

...

13595 : witch = 20 and min dis = 1.3417912 -> 1d2eb83c148bac438af0ad37e12d0871.jpg label is ZJL221
13596 : witch = 4 and min dis = 1.2878293 -> 1f14f10072777fe5b2a2bfb69ec855c5.jpg label is ZJL205
13597 : witch = 28 and min dis = 0.9746412 -> 1f7a9d2625e9aa955220647a72e10e48.jpg label is ZJL229
13598 : witch = 4 and min dis = 1.5444953 -> 20449ba7e11cdfe4b6ca81e8150290fe.jpg label is ZJL205
13599 : witch = 4 and min dis = 1.491852 -> 21c0ef28e340b8bbc9d596aa7d5152b3.jpg label is ZJL205
13600 : witch = 4 and min dis = 1.2212929 -> 240a5d5605719788bc2c948ba4b574d1.jpg label is ZJL205
13601 : witch = 4 and min dis = 1.0884511 -> 25354b6e2da24425325c6781b9ddf5be.jpg label is ZJL205
13602 : witch = 4 and min dis = 1.2173768 -> 27d9ebdb97f9467464c75b52d51f6540.jpg label is ZJL205
13603 : witch = 20 and min dis = 1.4977645 -> 2829f1dec938feb28c727236745e3459.jpg label is ZJL221
13604 : witch = 4 and min dis = 0.9259491 -> 284a4225a3e5557ae0c78881a77c08c6.jpg label is ZJL205
13605 : witch = 4 

13845 : witch = 4 and min dis = 1.2359393 -> f9e5489a9217fbc82b7b67e5b8cee2cb.jpg label is ZJL205
13846 : witch = 4 and min dis = 1.2223682 -> fa9a91ce508d5ce0c0382ef0d3dbe52f.jpg label is ZJL205
13847 : witch = 32 and min dis = 1.473589 -> faf68afc1aebde6fdd330be6867c9ac4.jpg label is ZJL233
13848 : witch = 37 and min dis = 1.4093853 -> fff5cfaffd576808793bcba209ad864a.jpg label is ZJL238
13849 : witch = 4 and min dis = 1.342525 -> 0054e5f8deed88b46af023ec45d95ee2.jpg label is ZJL205
13850 : witch = 4 and min dis = 1.2807345 -> 005f84dc5da39b9c7aad29636a61123c.jpg label is ZJL205
13851 : witch = 4 and min dis = 1.3635947 -> 00a09fe64b9427c0cde7ce8bdd2186bc.jpg label is ZJL205
13852 : witch = 4 and min dis = 1.1672512 -> 00e6f3a16bed8fc5e020dc21ad1e1f7f.jpg label is ZJL205
13853 : witch = 4 and min dis = 1.1459324 -> 016539459924e853a2a052dbe6ca909b.jpg label is ZJL205
13854 : witch = 4 and min dis = 1.0461224 -> 024c78a9dd7d77b0e4f047aa36421df2.jpg label is ZJL205
13855 : witch = 4 an

14094 : witch = 4 and min dis = 1.1756392 -> 80829834d243b2d3cd3f55d65011b60c.jpg label is ZJL205
14095 : witch = 4 and min dis = 1.3056746 -> 824c3b63db90f8d0c99575a582f9bca5.jpg label is ZJL205
14096 : witch = 4 and min dis = 1.0869552 -> 82f6483dab0ed066dac8bae7e88c8151.jpg label is ZJL205
14097 : witch = 4 and min dis = 1.224816 -> 8525c26e1784b1ebd6d42736dd744722.jpg label is ZJL205
14098 : witch = 4 and min dis = 1.2038621 -> 85327782a7fe92efafd7f2dfbe7ba421.jpg label is ZJL205
14099 : witch = 4 and min dis = 1.1904485 -> 8589ad267950cdb21116481f2745e68f.jpg label is ZJL205
14100 : witch = 4 and min dis = 1.2643164 -> 86b894bd697c3085808aae969e73cc4d.jpg label is ZJL205
14101 : witch = 4 and min dis = 1.1709045 -> 8782ee5b311f6015bdba433caa57895d.jpg label is ZJL205
14102 : witch = 4 and min dis = 1.303534 -> 881fa899dee30bd4c825682d4d65676e.jpg label is ZJL205
14103 : witch = 4 and min dis = 1.3880012 -> 88692b5014c627ff9fabd5e151120ae5.jpg label is ZJL205
14104 : witch = 4 and 

14344 : witch = 4 and min dis = 1.2748865 -> 1b43dcef7eefd245427cb47532fb8682.jpg label is ZJL205
14345 : witch = 4 and min dis = 1.2743552 -> 1b7f2c8aafe4d03714dfc0944fe41c4a.jpg label is ZJL205
14346 : witch = 4 and min dis = 1.2832836 -> c2ba395214b78123b7baff2f96154848.jpg label is ZJL205
14347 : witch = 4 and min dis = 1.4058622 -> 532b5a5177640a54e636461edc5b7de4.jpg label is ZJL205
14348 : witch = 28 and min dis = 0.77853465 -> 018fbec9d845b6a3c8929dc8d3561ce4.jpg label is ZJL229
14349 : witch = 4 and min dis = 1.0973969 -> 4b7dbbb6c54ebb215ffc76a385271b09.jpg label is ZJL205
14350 : witch = 4 and min dis = 1.3051066 -> 7cb9b6ac2bc946ad96df00b9e0c79c56.jpg label is ZJL205
14351 : witch = 4 and min dis = 1.3883215 -> a45a3c484577cdae7a5d3d63600b5ce4.jpg label is ZJL205
14352 : witch = 4 and min dis = 1.014715 -> ccbe14f63ebbf009468be52cb8949444.jpg label is ZJL205
14353 : witch = 4 and min dis = 1.256502 -> e23c4be8c3cc84462e96b7a693ea56d8.jpg label is ZJL205
14354 : witch = 4 an

14594 : witch = 28 and min dis = 0.74836034 -> 6dc6f0d96909cd798c62f1851a1e2204.jpg label is ZJL229
14595 : witch = 28 and min dis = 0.9463171 -> e43f1e42ae90921a13ab169e28414836.jpg label is ZJL229
14596 : witch = 1 and min dis = 1.2625861 -> 5c74fb235e023497f8dac340bdcc3a55.jpg label is ZJL202
14597 : witch = 28 and min dis = 0.51457953 -> 5f2a667f4ed9044d9bacc07127a17efc.jpg label is ZJL229
14598 : witch = 28 and min dis = 0.81381416 -> f89a952cf7175f9a13bc55cb49911bd0.jpg label is ZJL229
14599 : witch = 28 and min dis = 0.7134542 -> 7d8fe8516f31da726f03c9dfa9d9734e.jpg label is ZJL229
14600 : witch = 28 and min dis = 0.974581 -> 8f3c273ef6007b1164ec76b9a317c935.jpg label is ZJL229
14601 : witch = 28 and min dis = 0.9602001 -> e49598ff9d7828483d4d50601c37ffa2.jpg label is ZJL229
14602 : witch = 4 and min dis = 1.0143591 -> 63c4bae2f8100829a39e472e2698c872.jpg label is ZJL205
14603 : witch = 28 and min dis = 0.6982138 -> 893fc4420ac0abd9a0bf2df0f1a04174.jpg label is ZJL229
14604 : wi

In [ ]:
from keras.applications.mobilenet import MobileNet
from keras.applications.resnet50 import ResNet50
from keras.applications.densenet import DenseNet121
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten
from keras import backend as K
from keras.layers import Input 
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
import os
from keras.callbacks import ModelCheckpoint

#InResNetV2Dir = '/home/holmes/keras_model/inception_resnet_v2/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5'
trainDataDir = currentDir + '/trainSet'

classes = ['ZJL169', 'ZJL103', 'ZJL190', 'ZJL104', 'ZJL122',
                        'ZJL45', 'ZJL61', 'ZJL48', 'ZJL120', 'ZJL195',
                        'ZJL182', 'ZJL152', 'ZJL125', 'ZJL22', 'ZJL3', 'ZJL111',
                        'ZJL99', 'ZJL177', 'ZJL31', 'ZJL157', 'ZJL146', 'ZJL55',
                        'ZJL171', 'ZJL109', 'ZJL26', 'ZJL150', 'ZJL36', 'ZJL145',
                        'ZJL101', 'ZJL30', 'ZJL138', 'ZJL19', 'ZJL166', 'ZJL73', 'ZJL32',
                        'ZJL160', 'ZJL85', 'ZJL175', 'ZJL88', 'ZJL186', 'ZJL87', 'ZJL142',
                        'ZJL12', 'ZJL106', 'ZJL29', 'ZJL84', 'ZJL126', 'ZJL105', 'ZJL14',
                        'ZJL131', 'ZJL129', 'ZJL67', 'ZJL149', 'ZJL139', 'ZJL200', 'ZJL164',
                        'ZJL4', 'ZJL180', 'ZJL196', 'ZJL28', 'ZJL184', 'ZJL137', 'ZJL178',
                        'ZJL124', 'ZJL6', 'ZJL191', 'ZJL102', 'ZJL57', 'ZJL49', 'ZJL1', 'ZJL95',
                        'ZJL78', 'ZJL113', 'ZJL165', 'ZJL108', 'ZJL192', 'ZJL123', 'ZJL42',
                        'ZJL77', 'ZJL144', 'ZJL34', 'ZJL92', 'ZJL179', 'ZJL188', 'ZJL58',
                        'ZJL75', 'ZJL66', 'ZJL81', 'ZJL93', 'ZJL63', 'ZJL65', 'ZJL72', 'ZJL70',
                        'ZJL154', 'ZJL96', 'ZJL163', 'ZJL94', 'ZJL162', 'ZJL130', 'ZJL38', 'ZJL15',
                        'ZJL121', 'ZJL5', 'ZJL198', 'ZJL53', 'ZJL71', 'ZJL18', 'ZJL90', 'ZJL156',
                        'ZJL13', 'ZJL159', 'ZJL23', 'ZJL153', 'ZJL11', 'ZJL97', 'ZJL167', 'ZJL185',
                        'ZJL140', 'ZJL127', 'ZJL187', 'ZJL199', 'ZJL158', 'ZJL128', 'ZJL147', 'ZJL76',
                        'ZJL60', 'ZJL119', 'ZJL83', 'ZJL143', 'ZJL133', 'ZJL24', 'ZJL197', 'ZJL173',
                        'ZJL51', 'ZJL47', 'ZJL52', 'ZJL44', 'ZJL69', 'ZJL172', 'ZJL181', 'ZJL100', 'ZJL62',
                        'ZJL141', 'ZJL40', 'ZJL110', 'ZJL115', 'ZJL9', 'ZJL174', 'ZJL89', 'ZJL91', 'ZJL170',
                        'ZJL16', 'ZJL107', 'ZJL151', 'ZJL68', 'ZJL21', 'ZJL135', 'ZJL2', 'ZJL10', 'ZJL37',
                        'ZJL25', 'ZJL118', 'ZJL43', 'ZJL59', 'ZJL35', 'ZJL183', 'ZJL79', 'ZJL193', 'ZJL80',
                        'ZJL54', 'ZJL50', 'ZJL39', 'ZJL117', 'ZJL8', 'ZJL56', 'ZJL116', 'ZJL7', 'ZJL114',
                        'ZJL98', 'ZJL194', 'ZJL168', 'ZJL46', 'ZJL132', 'ZJL82', 'ZJL64', 'ZJL189', 'ZJL41',
                        'ZJL86', 'ZJL176', 'ZJL161']

#base_model = MobileNet(weights = None, include_top = False, pooling = 'avg' )
base_model = DenseNet121( weights = None, include_top = False, pooling='avg' )
base_modelOutput = base_model.output

#x = Flatten()( base_modelOutput )
x = Dense(1024, activation = 'relu')( base_modelOutput )
predictions = Dense(190, activation = 'softmax')( x )
model = Model(inputs = base_model.input, outputs = predictions)
model.summary()
'''
for layer in base_model.layers:
    layer.trainable = False
'''
model.compile( loss = 'categorical_crossentropy', optimizer = SGD(lr = 0.01, momentum = 0.9), metrics = ['accuracy'] )

train_dataGen = ImageDataGenerator( rescale = 1. / 255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True )
#val_dataGen = ImageDataGenerator( rescale = 1. / 255 )



modelDir = currentDir + '/model'

if not os.path.exists( modelDir ):
    os.mkdir( modelDir )

modelDir = currentDir + '/model/DenseNet121-ep{epoch:03d}-loss{loss:.5f}-acc{acc:.5f}.h5'
checkpoint = ModelCheckpoint( modelDir, monitor='loss', verbose=1, save_best_only= True, save_weights_only=False, mode='min', period=1 )

trainGenerator =  train_dataGen.flow_from_directory( trainDataDir, target_size = (224, 224), batch_size = 16, classes = classes )
#valGenerator = val_dataGen.flow_from_directory( valDataDir, target_size = (299, 299), batch_size = 4, shuffle = False, classes = ['cat', 'dog']  )
#model.fit_generator( trainGenerator, steps_per_epoch = 4375, epochs = 10, verbose = 1, validation_data = valGenerator, validation_steps = 1875 )
model.fit_generator( trainGenerator, steps_per_epoch = 3000, epochs = 200, verbose = 1,callbacks=[checkpoint] )
modelDir = currentDir + '/model/DenseNet121-lastepoch.h5'
model.save( modelDir )

In [32]:
from keras.applications.densenet import DenseNet121
from keras.models import load_model
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten
from keras import backend as K
from keras.layers import Input 
import os

os.environ['CUDA_VISIBLE_DEVICES'] = "0"

base_model = DenseNet121( weights = None, include_top = False, pooling='avg' )
base_modelOutput = base_model.output
#x = Flatten()( base_modelOutput )
x = Dense(1024, activation = 'relu')( base_modelOutput )
predictions = Dense(190, activation = "softmax")( x )
model = Model(inputs = base_model.input, outputs = predictions)
model.summary()

"""
base_model = load_model(model_dir + model_name + '_baseline_model.h5')
model = Model(inputs=base_model.input,
                  outputs=base_model.get_layer(model_feature_layer).output)
"""

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_5 (ZeroPadding2D (None, None, None, 3 0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, None, None, 6 9408        zero_padding2d_5[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, None, None, 6 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

"\nbase_model = load_model(model_dir + model_name + '_baseline_model.h5')\nmodel = Model(inputs=base_model.input,\n                  outputs=base_model.get_layer(model_feature_layer).output)\n"

In [33]:
model.load_weights("./models/densenet121/DenseNet121c190_baseline_model.h5")

In [55]:
from keras.preprocessing.image import ImageDataGenerator

testDataDir = "/home/anhaoran/data/zero-shot-tianchi/dataset_A/test"
data_generator = ImageDataGenerator( rescale = 1. / 255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True )
test_generator = data_generator.flow_from_directory( testDataDir, target_size = (224, 224), batch_size = 16, class_mode = None, shuffle = False)

Found 14633 images belonging to 1 classes.


In [ ]:
print()

In [56]:
predictions_generator = model.predict_generator(test_generator)
print(predictions_generator.shape)
print(predictions_generator)

(14633, 190)
[[2.6051505e-27 3.7624236e-27 1.0000000e+00 ... 6.3801713e-23
  1.2469516e-25 3.0398540e-21]
 [2.0462483e-14 2.1852661e-17 7.0042599e-22 ... 2.7408216e-14
  1.6850164e-10 8.8367763e-13]
 [6.6731283e-16 3.2545851e-21 5.2006649e-24 ... 3.4459816e-10
  8.4094675e-17 3.1677868e-15]
 ...
 [1.1289058e-05 9.1608263e-06 6.6066990e-18 ... 2.5741033e-14
  2.4912906e-08 2.0152004e-01]
 [6.5107560e-14 3.3525591e-16 7.6538548e-23 ... 3.1894127e-13
  3.9203925e-15 4.7866471e-15]
 [3.9877310e-19 1.2421672e-23 1.0133407e-22 ... 4.4882324e-19
  7.6669423e-24 2.8489392e-24]]


# Array

In [16]:
import numpy as np
from keras.preprocessing import image

data_dict = {}
data_test = open("/home/anhaoran/data/zero-shot-tianchi/dataset_A/test/image.txt")
data_test = data_test.readlines()#[:2]
length = len(data_test)
for i in range(length):
    data_dict.update({data_test[i][:-1] : "test"})
print("The number of test samples is : ", len(data_dict))


path_test = r'/home/anhaoran/data/zero-shot-tianchi/dataset_A/test/test/'
length = len(data_dict)
data_x = np.zeros((length, 224, 224, 3))
idx = 0
for img_name, img_label in data_dict.items():
    img = image.load_img(path_test + img_name, target_size=(224, 224, 3))
    data_x[idx] = image.img_to_array(img)
    idx += 1

The number of test samples is :  14633


In [17]:
print(data_x)

[[[[ 85.  82.  65.]
   [ 85.  82.  65.]
   [ 85.  82.  65.]
   ...
   [ 80.  62.  58.]
   [ 80.  62.  58.]
   [ 80.  62.  58.]]

  [[ 85.  82.  65.]
   [ 85.  82.  65.]
   [ 85.  82.  65.]
   ...
   [ 80.  62.  58.]
   [ 80.  62.  58.]
   [ 80.  62.  58.]]

  [[ 85.  82.  65.]
   [ 85.  82.  65.]
   [ 85.  82.  65.]
   ...
   [ 80.  62.  58.]
   [ 80.  62.  58.]
   [ 80.  62.  58.]]

  ...

  [[ 37.  41.  40.]
   [ 37.  41.  40.]
   [ 37.  41.  40.]
   ...
   [ 74.  89.  94.]
   [ 74.  89.  94.]
   [ 74.  89.  94.]]

  [[ 37.  41.  40.]
   [ 37.  41.  40.]
   [ 37.  41.  40.]
   ...
   [ 74.  89.  94.]
   [ 74.  89.  94.]
   [ 74.  89.  94.]]

  [[ 37.  41.  40.]
   [ 37.  41.  40.]
   [ 37.  41.  40.]
   ...
   [ 74.  89.  94.]
   [ 74.  89.  94.]
   [ 74.  89.  94.]]]


 [[[ 91. 118.  67.]
   [ 91. 118.  67.]
   [ 91. 118.  67.]
   ...
   [ 64.  75.  71.]
   [ 64.  75.  71.]
   [ 64.  75.  71.]]

  [[ 91. 118.  67.]
   [ 91. 118.  67.]
   [ 91. 118.  67.]
   ...
   [ 64.  75.  71.]
 

In [18]:
data_x /= 255.0
print(data_x)

[[[[0.33333333 0.32156863 0.25490196]
   [0.33333333 0.32156863 0.25490196]
   [0.33333333 0.32156863 0.25490196]
   ...
   [0.31372549 0.24313725 0.22745098]
   [0.31372549 0.24313725 0.22745098]
   [0.31372549 0.24313725 0.22745098]]

  [[0.33333333 0.32156863 0.25490196]
   [0.33333333 0.32156863 0.25490196]
   [0.33333333 0.32156863 0.25490196]
   ...
   [0.31372549 0.24313725 0.22745098]
   [0.31372549 0.24313725 0.22745098]
   [0.31372549 0.24313725 0.22745098]]

  [[0.33333333 0.32156863 0.25490196]
   [0.33333333 0.32156863 0.25490196]
   [0.33333333 0.32156863 0.25490196]
   ...
   [0.31372549 0.24313725 0.22745098]
   [0.31372549 0.24313725 0.22745098]
   [0.31372549 0.24313725 0.22745098]]

  ...

  [[0.14509804 0.16078431 0.15686275]
   [0.14509804 0.16078431 0.15686275]
   [0.14509804 0.16078431 0.15686275]
   ...
   [0.29019608 0.34901961 0.36862745]
   [0.29019608 0.34901961 0.36862745]
   [0.29019608 0.34901961 0.36862745]]

  [[0.14509804 0.16078431 0.15686275]
   [0.1

In [3]:
from keras.preprocessing.image import ImageDataGenerator

testDataDir = "/home/anhaoran/data/zero-shot-tianchi/dataset_A/test"
data_generator = ImageDataGenerator( rescale = 1. / 255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True )
test_generator = data_generator.flow_from_directory( testDataDir, target_size = (224, 224), batch_size = 16, class_mode = None, shuffle = False)
data_generator.fit(data_x)

Found 14633 images belonging to 1 classes.


In [4]:
print(data_x.shape)

(14633, 224, 224, 3)


In [5]:
from keras.applications.densenet import DenseNet121
from keras.models import load_model
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten
from keras import backend as K
from keras.layers import Input 
import os

os.environ['CUDA_VISIBLE_DEVICES'] = "0"

base_model1 = DenseNet121( weights = None, include_top = False, pooling='avg' )
base_modelOutput1 = base_model1.output
#x = Flatten()( base_modelOutput )
x1 = Dense(1024, activation = 'relu')( base_modelOutput1 )
predictions1 = Dense(190, activation = None)( x1 )
model1 = Model(inputs = base_model1.input, outputs = predictions1)
model1.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, None, None, 6 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, None, None, 6 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [12]:
model1.load_weights("./models/densenet121/DenseNet121c190_baseline_model.h5")
predictions = model1.predict(data_x)

In [19]:
predictions = model1.predict(data_x)
print(predictions.shape)
print(predictions)

(14633, 190)
[[-15.922118    11.275054   -25.29345    ...  -9.429393    11.288981
   26.834106  ]
 [ 19.12321    -17.002773    -1.1713786  ...   1.5334485    4.164734
    1.8102672 ]
 [ 17.90886      7.446069    14.155176   ...  -4.3788576    6.6282654
   -3.6674366 ]
 ...
 [  3.7819204    4.5087757   -1.509265   ...   4.635251     0.09191209
   17.26619   ]
 [  1.5649472   17.250904   -16.159262   ...  12.172527     8.475
   -0.6165206 ]
 [  1.5805846   14.490442   -12.515295   ... -18.486254     4.8094797
   -6.425019  ]]


In [20]:
print(data_x)

[[[[0.33333333 0.32156863 0.25490196]
   [0.33333333 0.32156863 0.25490196]
   [0.33333333 0.32156863 0.25490196]
   ...
   [0.31372549 0.24313725 0.22745098]
   [0.31372549 0.24313725 0.22745098]
   [0.31372549 0.24313725 0.22745098]]

  [[0.33333333 0.32156863 0.25490196]
   [0.33333333 0.32156863 0.25490196]
   [0.33333333 0.32156863 0.25490196]
   ...
   [0.31372549 0.24313725 0.22745098]
   [0.31372549 0.24313725 0.22745098]
   [0.31372549 0.24313725 0.22745098]]

  [[0.33333333 0.32156863 0.25490196]
   [0.33333333 0.32156863 0.25490196]
   [0.33333333 0.32156863 0.25490196]
   ...
   [0.31372549 0.24313725 0.22745098]
   [0.31372549 0.24313725 0.22745098]
   [0.31372549 0.24313725 0.22745098]]

  ...

  [[0.14509804 0.16078431 0.15686275]
   [0.14509804 0.16078431 0.15686275]
   [0.14509804 0.16078431 0.15686275]
   ...
   [0.29019608 0.34901961 0.36862745]
   [0.29019608 0.34901961 0.36862745]
   [0.29019608 0.34901961 0.36862745]]

  [[0.14509804 0.16078431 0.15686275]
   [0.1

In [ ]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    pass  # TODO: Compute and return softmax(x)
    x = np.array(x)
    x = np.exp(x)
    x.astype('float32')
    if x.ndim == 1:
        sumcol = sum(x)
        for i in range(x.size):
            x[i] = x[i]/float(sumcol)
    if x.ndim > 1:
        sumcol = x.sum(axis = 0)
        for row in x:
            for i in range(row.size):
                row[i] = row[i]/float(sumcol[i])
    return x

print(softmax(predictions[0]))
print(np.sum(softmax(predictions[0])))
print(np.max(softmax(predictions[0])))
print(np.argmax(softmax(predictions[0])))

# Done

In [36]:
print(predictions_generator[0])
print(np.sum(predictions_generator[0]))
print(np.max(predictions_generator[0]))
print(np.argmax(predictions_generator[0]))

[5.82096538e-10 3.38224097e-19 7.39871854e-19 3.15099734e-26
 3.89813517e-20 4.25133418e-20 9.65392842e-15 8.15243928e-10
 8.36911165e-20 4.23184717e-23 3.45585439e-17 7.84190074e-11
 1.85185328e-18 3.72760396e-15 8.69738797e-20 2.09242901e-11
 8.93505873e-17 7.87193461e-18 5.58982496e-22 2.30996753e-03
 2.09173628e-17 1.07657453e-15 1.00238054e-21 1.11527978e-21
 1.05985474e-24 9.01643282e-20 2.49153598e-15 2.58081837e-20
 6.28671117e-12 2.73722284e-21 1.41913636e-15 1.03036586e-25
 6.98012870e-20 1.14008623e-15 5.10886689e-12 1.08234263e-13
 4.37836707e-18 1.57129602e-04 2.13953218e-13 1.18346029e-14
 9.52179817e-20 2.52988531e-14 1.06559582e-15 1.48038365e-20
 6.91131616e-21 1.51219979e-18 8.27300471e-18 3.93003026e-11
 9.12567430e-20 5.13576421e-12 1.41331350e-21 1.80961221e-24
 1.93615784e-16 1.22922946e-14 3.67786924e-05 6.73343759e-15
 4.56740803e-19 4.23187620e-17 2.34346441e-25 9.12197394e-15
 2.88383854e-19 7.51675903e-13 1.73275396e-18 6.25236150e-14
 2.71991441e-13 1.020567

In [37]:
half_sum = 0
for i in range(len(predictions_generator)):
    if np.max(predictions_generator[i]) >= 0.5:
        print(np.max(predictions_generator[i]))
        half_sum += 1
print(half_sum, "in ", len(predictions_generator), "--->", half_sum * 1.0 / len(predictions_generator))

0.9974935
0.98872185
0.51551425
0.99997413
0.6444084
0.61996967
0.96063226
1.0
0.9999993
0.9991567
0.7950864
0.7359122
0.9142175
0.8902741
0.9998946
0.79746485
0.81898046
0.9997229
0.72258
0.7364734
0.9489371
0.992722
0.9996793
0.9999938
0.63353103
0.9813767
0.5269833
0.9601442
0.8696405
0.98148316
0.96854246
0.6467017
0.87614423
0.9979607
0.99997604
0.9805588
0.9999784
0.9993585
0.9981048
0.74060136
0.8280392
0.9780969
0.5053381
0.5184258
0.9597594
0.9999254
0.99981636
0.9238433
0.9870855
0.92994976
0.9986004
0.9951272
1.0
0.8894961
0.5239114
0.89356524
0.99896634
0.5430289
0.9092233
0.99999964
0.99998975
0.8741833
0.986382
0.9957111
0.9998054
0.9998872
0.66755706
0.8990557
0.87032956
0.9084275
0.9999033
1.0
0.9873964
0.99967265
0.99994683
0.7108768
0.9713136
1.0
0.9938023
0.57477486
0.9999999
0.99999976
0.9972023
1.0
0.65778095
0.9949649
0.8582507
1.0
0.8778059
0.9994796
0.99160856
0.893965
0.98895407
0.9975909
0.99999905
0.9878955
0.9999517
0.8652225
0.9999666
0.9573136
0.94006985
0

1.0
0.97987396
1.0
0.9762626
0.99999845
0.99895966
0.9991066
0.9999409
1.0
1.0
0.99999964
0.99905986
1.0
0.99999857
0.99646854
0.7800216
0.875952
0.9999999
1.0
0.9998522
0.94528973
1.0
0.9209752
0.9999864
0.9013044
0.9927071
0.66481036
0.9999162
0.993292
0.9872804
0.99619836
0.99949384
0.581874
0.8347327
0.99637586
0.98743236
0.9876987
1.0
0.99999905
0.75620097
0.9936405
0.60392636
1.0
0.86553854
0.93774277
1.0
0.9912039
0.9994741
0.9950369
0.9126765
0.9361221
0.7106096
0.9810901
0.61922795
0.87156487
0.86354613
1.0
0.59442
0.5675799
0.71786124
0.9800403
0.6213119
0.99024594
0.8575072
0.77507734
0.99963367
0.71416163
0.99419034
1.0
0.9999968
0.9377515
0.74083036
0.99910104
0.84271574
0.96321106
0.91688865
0.9838254
0.98531216
0.8320782
0.9997625
0.86918783
0.99999833
0.99526596
0.9976907
0.97632265
0.9999989
0.9159099
0.9999689
0.9997271
0.86849195
0.9996244
0.9857904
0.9995339
1.0
0.9602874
0.9999975
1.0
0.8242867
0.9449093
0.97855705
0.59840935
0.87881905
0.9791157
0.9989256
0.860968

0.789797
0.9124587
0.80323124
0.6274891
0.9999746
1.0
1.0
0.8559074
0.99999917
0.99999905
0.691851
0.91394335
0.9999999
0.9963387
0.97909206
1.0
0.5033067
0.9994505
0.7574927
0.50146526
0.7467546
0.99546117
0.98971516
0.8708781
0.9216989
0.90989166
0.99846774
0.99975663
0.97934854
0.9841613
0.9999639
0.9916774
0.54111594
0.978311
0.9981919
0.99910295
0.5728971
0.9868552
0.76985717
0.86640215
0.97672343
0.99876356
0.9953317
0.9993032
0.99999964
0.9050359
0.9963056
0.9991579
0.9999999
0.8677611
1.0
1.0
1.0
0.959946
0.9659503
0.9999455
0.999718
0.9995396
0.99999964
0.63004786
0.99997675
0.99614966
0.9863301
0.99999714
0.6972836
0.99961156
0.99005216
0.99909854
0.9997869
0.8741531
0.9515672
0.9762731
0.8179614
1.0
0.5886012
0.9210951
0.99960667
0.9985765
0.81056595
0.7651531
0.9999013
0.9215757
0.7271364
0.99976104
0.9912798
0.69760156
0.76882434
0.5230412
0.9380093
0.8871575
0.5299513
1.0
0.8570617
0.83644164
0.99514306
0.9336393
0.9407914
0.8642675
0.9999374
0.84218264
0.9926246
0.810427

0.6860945
0.9970812
0.9996493
0.9618378
0.9832386
0.9999995
0.9776559
0.96952975
0.968776
0.999979
0.9999925
0.86671656
0.986523
0.9508348
0.9999877
0.86412024
0.9584973
0.9373951
1.0
0.99998915
0.9910677
1.0
0.9887538
0.5051363
0.99485505
0.9961119
1.0
0.99947935
0.99811006
0.9134529
0.99561715
0.8278121
0.82859254
0.5886158
0.99638903
0.5150575
0.99991167
0.98655117
0.92652714
0.9926939
0.6358515
0.93405753
0.9999678
1.0
0.81321704
0.9991404
0.87696135
1.0
0.99885285
0.539255
0.95884633
0.9649796
0.99999976
0.9999927
0.999936
0.9362497
1.0
0.6591297
0.6509757
0.9999999
0.9999993
0.98524064
0.9999999
0.9406228
0.99113125
0.9999852
0.93009645
0.9931503
0.8155165
0.7352348
0.5506238
0.74423385
0.51524293
1.0
1.0
0.5994975
0.9998043
0.97668946
0.9906346
0.9999969
0.7819352
0.9999901
0.7323476
0.9999198
0.65923506
0.99744725
1.0
1.0
0.9823607
0.6503467
0.9997867
0.92742556
0.6629491
0.9780149
0.9464887
0.9893305
0.8281626
0.9999995
0.9289605
0.91588557
0.96203685
0.9999641
0.67084163
0.99

0.9814464
0.60469294
0.6837226
0.98253065
0.7151752
0.9895363
0.7932183
0.9992047
0.6648423
0.99291104
0.9999707
0.5113461
0.96699286
0.9720837
0.8122354
1.0
0.99335444
0.8472526
0.99695575
0.965656
0.99975973
0.9873663
1.0
0.9991134
0.9993793
0.52997214
0.8955405
1.0
0.99999976
0.71032
0.75942564
0.95503485
0.9823805
0.9994863
1.0
0.953655
0.99999917
0.9999933
0.5031883
0.99944335
0.7642625
0.96781844
0.9832882
1.0
0.96639496
1.0
1.0
0.9754868
0.9999832
0.9712762
0.7984927
0.90468234
0.8018167
0.7489867
0.983471
0.8842789
0.9917504
0.57867146
1.0
0.9881958
0.99997246
0.9971751
0.9976616
0.91871583
0.9522012
0.55966103
0.58019626
0.81203693
0.9997433
0.5522428
0.9998363
0.90147924
0.99999917
0.96487194
1.0
0.97352874
1.0
1.0
0.99775
0.8139444
1.0
0.99988556
0.8991684
0.71261877
0.9561442
0.67112577
0.9994629
0.61968774
0.9999945
0.9159881
0.82570994
0.99985623
0.9999995
0.9981018
0.9974502
0.9478727
0.99998283
0.9995639
0.96693945
1.0
1.0
0.99905986
0.73378575
0.78201413
0.9998282
0.97

...

0.9904614
0.6505174
0.8979455
0.99994063
1.0
0.99962974
0.90287316
0.9062832
0.97444916
0.9999286
0.6845171
0.99755657
0.9693132
0.9983485
0.6829551
0.99858403
0.9969999
0.90976006
0.96523464
0.9997795
0.77867156
1.0
0.6241489
0.9999926
0.9550867
0.99575824
0.6524791
0.999948
0.9083085
0.5773346
1.0
0.64063627
0.999998
0.99999976
0.709948
0.56054145
1.0
0.55206937
1.0
0.6337198
0.86829543
0.6010797
0.92704767
1.0
0.6064598
0.99992526
0.81931424
1.0
0.8946288
0.9999995
0.9995932
0.99579096
0.9975503
0.9947325
0.9985959
0.54606515
0.9901416
1.0
0.6672742
0.6538205
0.86813027
0.97339565
0.9975943
0.8367101
0.9303133
0.99697185
0.9888826
0.9999999
0.98874235
0.996959
0.99999714
1.0
0.9992471
0.91486585
0.851232
0.99914145
0.7119144
0.5703131
0.9906109
0.96390337
0.9999937
0.99978393
0.98252755
0.99982256
0.99997294
0.9434512
0.84187204
0.9485084
0.9559879
0.9992054
1.0
0.99999976
1.0
0.6563976
1.0
0.78458047
0.9924204
0.99914026
0.8133353
0.59273607
0.7544585
0.99164116
0.8952724
0.9290039

0.99598336
0.9740815
0.5600458
0.99967957
0.99334794
0.9906164
0.7665173
0.9994906
0.54148954
0.9871378
0.9999993
0.9839754
0.92417705
1.0
0.99998534
0.9983242
1.0
0.9999989
0.9919218
0.99522793
0.69167024
1.0
0.8398926
1.0
0.9986457
0.9956619
0.9645159
0.58975184
0.78344095
0.9468782
1.0
0.78918016
0.99995863
0.80183065
0.9857735
0.9999788
0.74781907
0.9737752
0.9989379
0.9801313
0.99997365
0.9932336
0.55715597
0.9999993
0.7826946
0.9046871
0.9972914
0.67062557
0.55860746
0.70793355
0.99458635
0.9996117
0.76886606
0.9298207
0.9999764
0.72167724
0.7711717
0.6860676
0.87244785
0.9992956
1.0
1.0
0.9695105
0.993288
0.99952686
1.0
0.9955734
0.8920247
0.99999964
0.99963
0.9852683
1.0
0.98515594
0.99786854
0.7504677
0.99126476
0.99968207
0.96052647
0.8863679
0.99996674
0.9999999
0.99114496
0.99183863
0.9574455
0.590461
0.99997354
0.9198957
0.5437941
0.89604497
1.0
1.0
0.8660371
0.93472683
0.833032
0.5232603
0.9836174
0.99957913
0.6530041
1.0
0.9974632
0.88952494
0.8566799
0.99187696
0.938034

0.7377494
0.8741283
1.0
0.9984388
0.9999485
1.0
0.99835086
0.99992216
0.7106722
0.99422896
0.9959247
0.85727006
0.85038394
0.995378
0.5032385
0.98555404
0.99909174
0.99986064
0.9798869
1.0
0.99812657
0.8158247
0.98535234
1.0
0.99433196
1.0
0.80212766
0.6342052
1.0
0.9999931
0.9998505
0.8229294
0.53696615
1.0
0.80045307
0.9238439
0.9999988
0.96020496
0.9999739
0.6740015
0.63975495
0.50373346
0.79215854
0.8786931
0.5001817
0.79090774
0.99672997
0.9184412
0.6404813
0.97984684
1.0
0.96412194
0.999728
0.9997129
0.9889442
1.0
0.99999976
0.98612773
0.9917583
1.0
0.9050897
0.99971896
0.9999999
0.78668165
0.9996778
0.9451974
0.9997917
0.9999995
0.98448426
0.98344177
0.99997056
0.8906842
0.55118585
0.534262
0.7459099
0.8992967
0.8966915
0.99394673
0.9999989
0.7256963
0.97637355
0.9977921
0.99872786
0.99601775
0.7303018
0.5412558
0.85571146
1.0
0.9987464
0.9011561
0.92875487
0.9900052
0.9933789
0.9147952
0.94799656
0.9999999
0.9998454
0.9717462
0.9218475
0.99838305
0.80613405
0.56178445
1.0
0.872

0.9209848
0.96659476
0.5390882
0.8692041
0.70106155
0.99789494
0.58304113
0.5051467
0.9680597
0.99744284
0.81379485
0.9999176
0.9979584
0.9467613
0.57456976
0.9751022
1.0
0.9350959
0.99999964
0.9827084
0.6764587
0.549518
0.99999225
0.63284624
0.9368981
0.7521815
0.57145876
0.8046491
0.9381377
0.6177409
1.0
0.98920363
1.0
0.9987764
0.9302664
0.9034641
0.98767114
0.78535086
0.9999889
0.5343476
0.9335537
0.90314776
0.88761926
0.9818201
0.99064434
0.82130355
0.956892
0.5629338
0.9999968
0.99993026
0.9916733
0.8077742
0.7953776
0.85064864
0.97633135
0.97758764
0.9743216
0.57824236
0.9947588
0.9976681
1.0
1.0
1.0
0.999931
0.9355416
1.0
0.78580606
0.7777431
0.97706944
0.99664575
0.5640627
0.99955946
0.9825706
0.9328717
0.8383855
0.9999263
0.9644416
0.83321023
0.9528699
0.56316805
0.98541105
0.8020998
0.94993025
0.8446354
0.96934545
0.95975035
0.50573725
0.6007484
0.66088706
0.9999311
0.99999833
0.9887742
0.7656793
0.9999887
0.8754254
0.9999974
0.99244
0.99970394
0.9999999
0.6927921
0.6911847


0.997883
0.9999629
0.90713894
0.9999982
0.9999064
0.9754415
0.7266052
0.98366666
0.6784659
1.0
0.64884543
0.96400005
0.64660966
1.0
0.88035333
0.99359304
0.99998844
0.98391867
0.89709836
0.9999999
0.9999994
0.86977047
1.0
0.9910953
0.91979384
0.9810586
0.99999595
0.6567808
0.9642402
0.98507833
1.0
0.9763914
0.99999964
0.9998667
0.99846244
0.9999895
0.99973685
0.99920493
0.72165984
0.9657769
1.0
0.8949524
0.99999785
0.99632335
1.0
0.9999999
0.9999845
0.975296
0.9960282
0.99896145
0.99998236
0.7093191
0.99999297
0.9820693
0.99931264
0.9807232
0.8548177
0.7905253
0.79542553
0.93790823
0.9999988
0.9999981
0.8827217
0.99901986
0.9997508
0.9999999
0.615152
0.90168023
0.98854476
1.0
1.0
0.9886092
0.87967324
0.99728453
0.7778998
0.82657695
0.9376575
0.5933056
0.60571176
0.5262998
0.994447
0.99945873
0.9415518
0.83400595
0.9996158
0.9986545
0.9987998
0.9732721
0.9954229
0.8093943
0.9968268
0.60255736
0.9097787
1.0
0.99419445
0.9978967
0.999775
0.78651786
0.9611603
0.97340256
1.0
0.59811616
1.0


In [41]:
nine_sum = 0
for i in range(len(predictions_generator)):
    if np.max(predictions_generator[i]) >= 0.9:
        print(np.max(predictions_generator[i]))
        nine_sum += 1
print(nine_sum, "in ", len(predictions_generator), "--->", nine_sum * 1.0 / len(predictions_generator))

0.9974935
0.98872185
0.99997413
0.96063226
1.0
0.9999993
0.9991567
0.9142175
0.9998946
0.9997229
0.9489371
0.992722
0.9996793
0.9999938
0.9813767
0.9601442
0.98148316
0.96854246
0.9979607
0.99997604
0.9805588
0.9999784
0.9993585
0.9981048
0.9780969
0.9597594
0.9999254
0.99981636
0.9238433
0.9870855
0.92994976
0.9986004
0.9951272
1.0
0.99896634
0.9092233
0.99999964
0.99998975
0.986382
0.9957111
0.9998054
0.9998872
0.9084275
0.9999033
1.0
0.9873964
0.99967265
0.99994683
0.9713136
1.0
0.9938023
0.9999999
0.99999976
0.9972023
1.0
0.9949649
1.0
0.9994796
0.99160856
0.98895407
0.9975909
0.99999905
0.9878955
0.9999517
0.9999666
0.9573136
0.94006985
0.900404
0.98928845
0.9938875
0.9999995
0.97612584
1.0
0.9966126
1.0
0.99970716
0.91367346
0.99670863
1.0
0.99985766
0.99999964
0.99999964
0.9546918
0.99748117
0.9995202
0.9992939
0.9168093
0.9999167
0.9765871
0.9027566
0.9999691
0.9964277
0.99601644
1.0
1.0
0.999884
0.9264241
0.999998
0.92159426
0.9839002
0.9663608
0.95033836
0.9633812
1.0
0.99882

0.9999404
0.9651643
0.98421
0.9996972
0.9999999
0.977103
0.9998265
0.99999464
0.9348058
1.0
0.99890375
0.99999094
1.0
1.0
1.0
1.0
0.9916455
1.0
1.0
0.98752207
0.9947752
0.99999905
0.99999976
0.99999917
0.99823725
0.9893009
0.9886596
0.9999453
0.99998176
0.93539214
1.0
1.0
0.99343795
0.9932769
0.9999999
0.99991953
0.9999981
0.98249304
0.99999976
0.9931312
0.9995993
0.99945873
0.9714162
0.99895954
0.9756033
0.9635593
0.99999356
0.998703
0.99999976
1.0
0.9977093
0.9985317
1.0
0.99998677
0.99377644
0.9999976
1.0
0.9960377
0.9777705
0.9975292
0.9790759
1.0
0.99999297
0.98634654
0.99999976
0.98807794
1.0
0.98486286
0.98715913
0.9988343
0.99975234
0.968732
0.9994667
1.0
0.9615425
0.9964947
0.9801225
0.9998
0.9794408
0.9960317
0.9866747
0.96880853
1.0
0.9997092
0.9986767
0.9541378
1.0
0.96462595
1.0
1.0
0.9988305
0.9597163
0.999337
0.96086454
0.9999807
0.99643564
0.99998116
0.98913443
1.0
0.9637162
1.0
0.99999857
0.9955318
0.99425226
0.9952673
1.0
0.99443567
0.9994388
0.999688
0.9999999
0.9997

1.0
1.0
0.99905986
0.9998282
0.97036403
0.9759744
1.0
0.9875896
0.9980204
0.9834383
0.9763668
1.0
0.99213034
0.99999607
0.99999964
1.0
0.9999119
0.9999777
0.99977905
1.0
0.9157466
0.9937611
0.98459595
0.99617994
0.95530874
0.99998593
0.99998105
0.9998548
0.9999999
0.9999999
1.0
0.99881786
1.0
0.9864949
0.999998
0.92276037
0.9132423
0.94195324
0.9972644
0.9973143
0.999711
0.999603
0.9932249
0.97047746
0.9929149
0.9794841
0.9925276
0.97476655
0.93516994
0.95267045
0.9197334
0.9954057
1.0
1.0
0.9866411
0.9999999
0.99994016
1.0
0.9999993
0.99999976
0.98511964
0.9999684
0.99338245
0.9999422
0.997129
0.9915292
0.97401017
0.99999976
0.9027925
0.9326402
0.9049003
0.999977
0.99999964
0.9758882
0.98631763
1.0
0.998858
0.99986815
0.9999993
0.9940645
1.0
0.9957372
0.9987104
1.0
1.0
0.9133896
1.0
0.9973558
0.9973744
0.9999832
0.99950576
1.0
0.9617579
1.0
0.96514976
0.9973502
0.9786781
0.99058914
0.9980477
0.9774259
0.99756193
0.96297497
0.99966776
0.9729171
0.92862105
0.9887341
0.99609274
0.9966133

0.95353836
1.0
0.9992204
0.97607976
0.976846
0.99974054
0.9995565
0.90937287
0.9999714
1.0
1.0
0.99998224
0.9978725
0.99999595
1.0
0.97334224
1.0
0.9995016
0.9994155
0.99983084
0.99484974
0.9999672
1.0
0.9071676
1.0
1.0
0.97454304
0.9789274
0.9999918
0.9230324
1.0
0.9999795
0.9351466
0.9876324
1.0
0.99992454
0.9999981
0.99643904
1.0
0.9998771
0.996086
0.9650392
0.9893917
0.99975497
0.99999785
1.0
0.9512124
0.99999845
0.977427
0.94478863
0.98563635
1.0
0.9920081
1.0
0.9877372
0.9982095
0.9993093
0.9999707
0.999997
0.9983261
0.9999981
1.0
0.9918133
0.9725681
0.9240328
0.9816046
0.99999976
0.9999932
1.0
0.9985662
0.9955415
0.99994314
1.0
0.9882252
0.96469194
0.9259411
0.9978371
0.9999993
1.0
0.97683674
0.99972934
0.96585345
0.9918549
1.0
0.9967223
0.9954464
0.9999995
0.93978804
0.99512154
0.9726449
0.99999964
0.976991
0.99639785
0.9999738
1.0
0.993196
0.9370953
0.98888165
0.9646914
0.9979797
0.99949
0.98332316
0.9901528
1.0
0.9992399
0.9992779
1.0
0.9372987
0.9997162
0.96361935
0.9075424


0.95827127
0.9999821
1.0
0.97705746
0.9444868
0.99999976
1.0
1.0
0.99992824
0.9789176
1.0
0.99976474
0.99021214
0.9999149
0.99989104
0.9970759
0.9999969
0.93889016
1.0
0.99993765
0.9881114
0.9998952
0.999814
0.9998605
0.91066325
1.0
0.9999416
0.9995408
0.99998474
0.9967512
0.9999918
0.99998415
1.0
0.99985456
0.98901945
0.99414575
0.944762
0.947571
0.9997781
0.9386153
0.9999975
0.9999927
0.99850047
1.0
0.9998735
0.950912
0.9999906
0.99999917
0.9131077
0.906732
0.999681
0.99393636
0.99341536
0.9957663
0.9616165
0.9442086
0.99998903
1.0
1.0
0.99500895
0.9404976
0.99999964
0.99924195
0.99500066
0.9999131
0.9772744
1.0
0.99999857
0.90178597
0.9517059
0.99317306
1.0
0.95458364
0.98365
0.952134
0.9068083
0.9745564
1.0
1.0
0.9983981
0.9931799
0.99999976
0.97202516
0.999972
0.99997556
0.9982495
0.99999976
0.9935807
0.96732485
0.9999238
0.99999595
0.99435544
0.99994516
1.0
0.98896575
0.95349
1.0
0.9954157
1.0
0.9981312
0.9964683
0.97363245
0.9953099
0.9999995
0.99960726
0.9987821
0.9575656
0.999

0.92704767
1.0
0.99992526
1.0
0.9999995
0.9995932
0.99579096
0.9975503
0.9947325
0.9985959
0.9901416
1.0
0.97339565
0.9975943
0.9303133
0.99697185
0.9888826
0.9999999
0.98874235
0.996959
0.99999714
1.0
0.9992471
0.91486585
0.99914145
0.9906109
0.96390337
0.9999937
0.99978393
0.98252755
0.99982256
0.99997294
0.9434512
0.9485084
0.9559879
0.9992054
1.0
0.99999976
1.0
1.0
0.9924204
0.99914026
0.99164116
0.9290039
0.9669652
1.0
0.99999344
0.9979547
0.99890566
0.9999758
0.9958276
0.96697927
0.9928282
1.0
0.9999999
1.0
0.92568874
0.9512619
0.99959296
0.9789786
0.9152504
1.0
0.9992417
0.99997926
0.9996014
0.9888708
0.999838
0.9999924
1.0
0.9857244
0.9973822
0.99999964
0.9983543
1.0
0.99992955
0.9867134
0.99461776
0.99561703
0.9999999
1.0
1.0
0.98610663
0.9996233
0.98738223
0.9999871
0.98087054
0.94588155
0.9999993
0.93830836
1.0
0.9172783
0.9917316
0.9209174
1.0
1.0
1.0
0.99877816
0.9999988
1.0
1.0
0.99981135
0.9854637
1.0
0.9409707
0.9396677
0.9999927
1.0
0.9999987
0.92156863
0.99834824
0.93

0.9981806
0.9808301
0.99996626
0.9999995
0.9999993
0.9570087
0.9999999
0.9992913
0.9393801
1.0
1.0
0.9999999
0.98027396
0.9371144
0.97939545
0.99999905
1.0
0.944752
0.9999999
1.0
0.9956176
0.9286452
0.9999988
0.9999753
0.98503184
0.9929717
0.97005904
0.93437535
0.99185985
0.97287333
1.0
0.9480215
0.991165
1.0
0.9995827
0.99999976
0.99155444
0.9871646
0.99732554
0.9465472
0.997841
1.0
0.99999523
0.99998736
0.99911135
0.9987166
0.94719946
0.9739171
0.99991643
1.0
1.0
0.9999565
0.91209424
0.978161
1.0
0.99989474
0.99940705
0.9318346
0.996869
0.9193431
0.98437977
0.9563411
0.9999597
1.0
0.9978175
0.9685385
0.99997187
0.9999869
0.9999919
0.99928856
0.99437827
1.0
0.95101815
0.97165966
0.99999917
0.9999968
0.9795681
0.99172217
0.9994374
0.9865957
0.91369516
1.0
0.99990463
0.99794644
0.9284057
0.9994869
1.0
0.98029375
0.98335403
0.9991511
0.99734735
0.9910164
0.95776975
0.99678683
0.99981636
0.9964259
0.999949
0.9998124
0.99999034
0.91171384
0.99999356
0.99983823
0.9969297
0.9947078
0.9918192

0.9998078
0.9998795
0.9423799
0.99957925
1.0
1.0
0.9988826
0.94986486
0.97986674
0.94376266
0.95760673
0.99731785
1.0
0.9971227
1.0
0.9996587
0.9988337
0.99995446
0.98291403
0.998585
0.91500354
0.99995303
0.9024856
1.0
0.9999151
0.9974396
0.9999949
0.97938615
0.9855215
0.9469388
0.99916756
0.9999877
0.967585
1.0
0.9931765
0.92923295
0.99999785
0.99981946
1.0
0.98908
0.99999845
0.98478425
0.99680066
0.9824223
0.9984681
0.99994755
0.9933821
0.99227047
0.9850964
1.0
0.9530043
0.9928899
0.9962727
0.999995
0.9996269
0.98700863
1.0
0.9999989
0.9992718
0.9999999
0.9999045
0.98899686
0.9901433
0.99696296
0.99939406
0.9697063
0.9835201
0.95133215
0.9998267
0.9389527
1.0
0.9665045
0.97557414
0.9999956
0.9988625
0.9999945
0.99174553
0.9999989
1.0
0.9997154
0.99903524
0.9962842
0.99960965
1.0
1.0
0.9883725
0.9999993
0.99768186
0.99999356
0.98043203
1.0
0.99864036
0.9999701
0.99995184
0.990914
0.98996943
0.9698545
0.999931
0.99691343
1.0
1.0
0.9586173
0.98012954
0.9980736
0.9999999
0.9961505
0.9999

1.0
0.91773087
0.9983682
0.99995637
0.99929845
1.0
0.9900857
0.9829958
1.0
0.9621432
0.99992657
0.9981736
0.9944186
0.99616313
0.9996525
0.94986475
0.9427248
0.90177804
0.9999814
1.0
0.999992
0.9999912
0.99991834
0.9999999
0.92659163
0.99996936
0.99781215
0.99999595
0.9931657
0.9996505
0.997883
0.9999629
0.90713894
0.9999982
0.9999064
0.9754415
0.98366666
1.0
0.96400005
1.0
0.99359304
0.99998844
0.98391867
0.9999999
0.9999994
1.0
0.9910953
0.91979384
0.9810586
0.99999595
0.9642402
0.98507833
1.0
0.9763914
0.99999964
0.9998667
0.99846244
0.9999895
0.99973685
0.99920493
0.9657769
1.0
0.99999785
0.99632335
1.0
0.9999999
0.9999845
0.975296
0.9960282
0.99896145
0.99998236
0.99999297
0.9820693
0.99931264
0.9807232
0.93790823
0.9999988
0.9999981
0.99901986
0.9997508
0.9999999
0.90168023
0.98854476
1.0
1.0
0.9886092
0.99728453
0.9376575
0.994447
0.99945873
0.9415518
0.9996158
0.9986545
0.9987998
0.9732721
0.9954229
0.9968268
0.9097787
1.0
0.99419445
0.9978967
0.999775
0.9611603
0.97340256
1.0


In [42]:
nine2_sum = 0
for i in range(len(predictions_generator)):
    if np.max(predictions_generator[i]) >= 0.99:
        print(np.max(predictions_generator[i]))
        nine2_sum += 1
print(nine2_sum, "in ", len(predictions_generator), "--->", nine2_sum * 1.0 / len(predictions_generator))

0.9974935
0.99997413
1.0
0.9999993
0.9991567
0.9998946
0.9997229
0.992722
0.9996793
0.9999938
0.9979607
0.99997604
0.9999784
0.9993585
0.9981048
0.9999254
0.99981636
0.9986004
0.9951272
1.0
0.99896634
0.99999964
0.99998975
0.9957111
0.9998054
0.9998872
0.9999033
1.0
0.99967265
0.99994683
1.0
0.9938023
0.9999999
0.99999976
0.9972023
1.0
0.9949649
1.0
0.9994796
0.99160856
0.9975909
0.99999905
0.9999517
0.9999666
0.9938875
0.9999995
1.0
0.9966126
1.0
0.99970716
0.99670863
1.0
0.99985766
0.99999964
0.99999964
0.99748117
0.9995202
0.9992939
0.9999167
0.9999691
0.9964277
0.99601644
1.0
1.0
0.999884
0.999998
1.0
0.99882215
0.99574643
0.9999869
1.0
0.99999726
0.9999924
0.9987857
0.9999993
0.9999999
1.0
0.9999604
0.9999845
1.0
0.99660623
0.9997775
0.99915636
1.0
1.0
0.9999987
0.99992573
1.0
0.9999981
0.99987173
0.9999783
0.9996921
1.0
1.0
0.9999827
0.99998903
1.0
1.0
0.9940163
0.9904484
1.0
0.99817026
0.99999976
0.99984753
0.9984151
0.9993717
0.9999964
0.9987772
1.0
0.9999926
0.99309385
1.0
0.9

0.99998105
1.0
1.0
0.9965043
0.99999976
0.9999906
0.9999975
0.99998724
0.9997323
0.99939716
0.9968515
0.99998355
0.99888414
0.9999045
1.0
0.99998605
0.9999871
0.9999826
0.99998295
0.9999999
0.998872
0.9999465
1.0
1.0
0.9999778
0.9999746
1.0
1.0
0.99999917
0.99999905
0.9999999
0.9963387
1.0
0.9994505
0.99546117
0.99846774
0.99975663
0.9999639
0.9916774
0.9981919
0.99910295
0.99876356
0.9953317
0.9993032
0.99999964
0.9963056
0.9991579
0.9999999
1.0
1.0
1.0
0.9999455
0.999718
0.9995396
0.99999964
0.99997675
0.99614966
0.99999714
0.99961156
0.99005216
0.99909854
0.9997869
1.0
0.99960667
0.9985765
0.9999013
0.99976104
0.9912798
1.0
0.99514306
0.9999374
0.9926246
1.0
0.99988365
1.0
0.9904186
0.99992144
0.9999993
1.0
0.9984035
0.9999907
1.0
0.99983144
0.9970577
1.0
0.99656004
0.99965847
0.9964933
1.0
0.99996626
0.99999106
0.9998791
1.0
0.99821436
0.9999728
0.9992944
0.9999999
1.0
0.9999999
0.99999976
0.9999988
1.0
0.9999863
1.0
0.99917847
0.99992514
0.9948828
1.0
0.9999894
0.998769
1.0
1.0
0.

1.0
0.9999119
0.9999777
0.99977905
1.0
0.9937611
0.99617994
0.99998593
0.99998105
0.9998548
0.9999999
0.9999999
1.0
0.99881786
1.0
0.999998
0.9972644
0.9973143
0.999711
0.999603
0.9932249
0.9929149
0.9925276
0.9954057
1.0
1.0
0.9999999
0.99994016
1.0
0.9999993
0.99999976
0.9999684
0.99338245
0.9999422
0.997129
0.9915292
0.99999976
0.999977
0.99999964
1.0
0.998858
0.99986815
0.9999993
0.9940645
1.0
0.9957372
0.9987104
1.0
1.0
1.0
0.9973558
0.9973744
0.9999832
0.99950576
1.0
1.0
0.9973502
0.99058914
0.9980477
0.99756193
0.99966776
0.99609274
0.9966133
1.0
0.99999416
0.9949609
0.9996501
0.99951994
0.9981515
0.9999999
0.99889225
0.9999999
1.0
0.9998528
0.99996674
0.99133885
0.9999999
0.9999405
1.0
0.9994685
0.99999964
1.0
0.9992067
1.0
0.9947429
0.9983209
0.990167
0.99998343
1.0
0.9999993
0.99901557
0.9999999
0.99951863
0.99999475
0.9907974
1.0
0.9984841
1.0
1.0
1.0
0.9985026
0.99823594
0.99983525
1.0
0.999892
1.0
1.0
1.0
1.0
0.9999627
1.0
0.99948525
0.99872166
0.9998791
0.99985147
0.99999

0.99999297
0.99579287
0.99999976
1.0
1.0
1.0
1.0
0.9999647
0.9997435
0.9999889
0.99999654
1.0
0.9992724
1.0
1.0
1.0
1.0
0.9991074
0.999926
0.99943477
0.9999926
0.9996805
0.99091655
0.9964258
0.9924648
1.0
0.9931063
1.0
1.0
0.99879044
0.99999785
0.99987924
0.99989665
0.99937373
0.9982514
1.0
1.0
0.99591714
0.999984
0.99040633
0.99999857
0.999995
0.99999785
0.99402714
0.99997413
0.99677366
0.9993806
1.0
1.0
0.99993837
0.9998845
1.0
0.999848
0.9999274
0.99809104
0.9999999
0.99992085
0.9996866
1.0
0.9959039
0.99995434
0.999974
0.99999034
0.9949604
1.0
1.0
0.99999976
1.0
0.99803203
0.99201435
1.0
0.9999324
0.9999968
1.0
1.0
0.9998437
0.999892
0.99999774
0.9983845
0.99987435
1.0
0.9999958
0.9967257
0.99976426
0.9999987
0.99889034
0.9999839
0.99682593
0.99983704
0.99788576
1.0
1.0
0.9996836
0.999997
0.999703
0.99712926
0.9944061
0.99809605
0.9985765
0.99999964
0.9981964
0.99999976
1.0
0.9979127
1.0
1.0
0.9976562
0.9969524
0.99923384
0.99999917
1.0
0.9987
0.99999845
0.9999993
0.9998271
0.99999

1.0
0.999998
0.99999976
1.0
1.0
1.0
0.99992526
1.0
0.9999995
0.9995932
0.99579096
0.9975503
0.9947325
0.9985959
0.9901416
1.0
0.9975943
0.99697185
0.9999999
0.996959
0.99999714
1.0
0.9992471
0.99914145
0.9906109
0.9999937
0.99978393
0.99982256
0.99997294
0.9992054
1.0
0.99999976
1.0
1.0
0.9924204
0.99914026
0.99164116
1.0
0.99999344
0.9979547
0.99890566
0.9999758
0.9958276
0.9928282
1.0
0.9999999
1.0
0.99959296
1.0
0.9992417
0.99997926
0.9996014
0.999838
0.9999924
1.0
0.9973822
0.99999964
0.9983543
1.0
0.99992955
0.99461776
0.99561703
0.9999999
1.0
1.0
0.9996233
0.9999871
0.9999993
1.0
0.9917316
1.0
1.0
1.0
0.99877816
0.9999988
1.0
1.0
0.99981135
1.0
0.9999927
1.0
0.9999987
0.99834824
1.0
0.99999976
0.9916248
0.99898034
0.9991209
0.992363
0.9994491
0.9999994
0.9929872
0.999946
0.99951744
0.9931252
0.9999999
0.999887
0.9999895
0.9999771
0.9980338
1.0
1.0
0.99507445
0.99999976
0.999788
0.99998736
0.9997986
0.9999528
1.0
0.99951994
0.99999976
0.9922127
1.0
1.0
0.99999976
0.9982644
0.99998

0.9987074
0.9999627
0.99963105
1.0
0.99495596
1.0
0.99827504
1.0
0.99999905
1.0
1.0
0.9998241
0.9999999
0.99969435
0.99999833
0.99724716
0.99960476
0.99960583
0.9997806
0.9999839
0.994961
0.99867487
1.0
0.9996414
0.9999647
0.99969983
0.999416
0.9994986
0.999371
1.0
0.9930093
0.99999964
0.99990165
0.992063
0.9980361
0.9999887
0.9992261
0.9978531
0.9934501
1.0
1.0
0.9988551
0.99746954
0.9999999
0.9999105
0.99946266
0.9999988
0.99999976
0.99999666
0.9999628
0.9999081
1.0
0.99984765
1.0
0.99946266
1.0
0.9943606
0.9997539
0.9997912
0.9999999
0.99994755
1.0
0.9994536
1.0
0.9928295
0.9999411
0.99999964
1.0
1.0
0.99999976
0.99999976
0.9999969
0.9941328
0.99349666
0.9990282
0.9998963
0.9988298
0.9971625
0.99987996
0.9996456
0.9999999
0.99982506
0.99999547
0.99999785
0.9996587
0.9999999
1.0
0.99633324
0.9998745
0.9999988
0.99667394
1.0
0.9986792
0.9999709
0.9999963
0.9986187
0.99959713
0.9952992
0.9999893
0.99995565
0.9999536
0.99996936
0.9990252
0.9998072
0.9995783
1.0
0.99587613
0.99999964
1.0

1.0
0.999488
0.9998485
0.9999995
0.998044
1.0
0.997778
1.0
1.0
0.99985147
0.99997926
1.0
1.0
0.9998417
0.99360955
0.9999981
0.9989969
0.99968946
0.9954678
0.9999926
0.99995005
0.99696463
0.9999691
0.9933729
0.9999391
1.0
0.9986834
0.9912663
0.9994399
0.99857104
0.9940646
0.99853075
0.99999344
0.99998045
1.0
0.99814785
0.99999976
0.9993364
0.9981256
1.0
0.9983682
0.99995637
0.99929845
1.0
0.9900857
1.0
0.99992657
0.9981736
0.9944186
0.99616313
0.9996525
0.9999814
1.0
0.999992
0.9999912
0.99991834
0.9999999
0.99996936
0.99781215
0.99999595
0.9931657
0.9996505
0.997883
0.9999629
0.9999982
0.9999064
1.0
1.0
0.99359304
0.99998844
0.9999999
0.9999994
1.0
0.9910953
0.99999595
1.0
0.99999964
0.9998667
0.99846244
0.9999895
0.99973685
0.99920493
1.0
0.99999785
0.99632335
1.0
0.9999999
0.9999845
0.9960282
0.99896145
0.99998236
0.99999297
0.99931264
0.9999988
0.9999981
0.99901986
0.9997508
0.9999999
1.0
1.0
0.99728453
0.994447
0.99945873
0.9996158
0.9986545
0.9987998
0.9954229
0.9968268
1.0
0.9941

In [44]:
nine5_sum = 0
for i in range(len(predictions_generator)):
    if np.max(predictions_generator[i]) >= 0.99999:
        print(np.max(predictions_generator[i]))
        nine5_sum += 1
print(nine5_sum, "in ", len(predictions_generator), "--->", nine5_sum * 1.0 / len(predictions_generator))

1.0
0.9999993
0.9999938
1.0
0.99999964
1.0
1.0
0.9999999
0.99999976
1.0
1.0
0.99999905
0.9999995
1.0
1.0
1.0
0.99999964
0.99999964
1.0
1.0
0.999998
1.0
1.0
0.99999726
0.9999924
0.9999993
0.9999999
1.0
1.0
1.0
1.0
0.9999987
1.0
0.9999981
1.0
1.0
1.0
1.0
1.0
0.99999976
0.9999964
1.0
0.9999926
1.0
0.99999714
1.0
0.99999976
1.0
0.9999918
0.999997
1.0
1.0
1.0
0.99999034
0.9999925
1.0
0.99999857
0.9999981
1.0
1.0
0.9999999
1.0
1.0
1.0
0.9999994
1.0
0.999995
0.9999995
0.9999988
0.9999925
0.9999994
0.9999957
0.99999917
1.0
1.0
1.0
0.99999964
0.9999999
0.99999666
0.99999654
0.9999932
1.0
1.0
0.9999994
1.0
0.99999964
1.0
1.0
0.99999976
0.9999994
0.9999999
0.9999962
1.0
0.9999943
1.0
1.0
0.9999987
0.99999917
0.9999963
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999
1.0
1.0
1.0
0.9999993
0.99999976
1.0
1.0
1.0
1.0
1.0
1.0
0.9999906
0.99999917
0.9999901
1.0
0.9999908
0.99999857
0.9999988
1.0
0.9999906
1.0
1.0
1.0
1.0
1.0
1.0
0.9999902
0.99999523
1.0
1.0
0.99999785
0.9999999
1.0
0.9999958
1.0
0.99999976
0.9999985

0.9999999
0.99999714
1.0
0.9999937
1.0
0.99999976
1.0
1.0
1.0
0.99999344
1.0
0.9999999
1.0
1.0
0.9999924
1.0
0.99999964
1.0
0.9999999
1.0
1.0
0.9999993
1.0
1.0
1.0
1.0
0.9999988
1.0
1.0
1.0
0.9999927
1.0
0.9999987
1.0
0.99999976
0.9999994
0.9999999
1.0
1.0
0.99999976
1.0
0.99999976
1.0
1.0
0.99999976
0.9999981
1.0
1.0
0.99999654
1.0
0.999997
1.0
1.0
1.0
0.9999987
0.9999995
0.9999988
0.9999944
1.0
1.0
0.9999995
0.9999968
1.0
1.0
1.0
0.99999917
0.9999989
0.9999999
1.0
1.0
1.0
1.0
0.99999976
0.9999937
1.0
1.0
1.0
0.9999976
1.0
0.99999344
0.99999845
0.99999666
1.0
1.0
0.9999962
1.0
0.9999982
0.9999933
1.0
0.9999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999931
0.9999999
1.0
0.9999982
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999993
0.9999976
0.99999535
0.99999905
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999958
1.0
0.99999905
1.0
0.99999774
1.0
0.9999956
0.9999995
1.0
0.9999962
1.0
1.0
1.0
0.9999956
1.0
1.0
1.0
0.9999963
1.0
0.9999943
0.99999905
0.9999999
1.0
0.9999919
1.0
1.0
1.0
1.0
0.9999943
0.9999999
0.

In [45]:
base_model = load_model('./models/densenet121/DenseNet121c190_baseline_model.h5')

In [46]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, None, None, 3 0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, None, None, 6 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, None, None, 6 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [65]:
import os
os.listdir(testDataDir + '/test')

['0d22af00b5813f0ff8dcc68c7e129e83.jpg',
 'da377cf14012aacea672c0c11f6b7f60.jpg',
 '2622469f468a2b9bea1bb967fc801a21.jpg',
 '1c0d7b4a4ee0ef008a5be5fa3c75ca50.jpg',
 '7bcf6b6580eca203e1f540e2b0b69f2a.jpg',
 '846db64da0a5c63668e9d7ac2bdf1b2b.jpg',
 'ff88baa78b642ac1aa519e0a399ffa3f.jpg',
 '64607298d907d8f21a6a777186716715.jpg',
 '784c99cacdc6b4a31c66218b4ff09ca3.jpg',
 '32b7a3046ef7fd1f49f441c702cce323.jpg',
 '3662de4f3db92d5892b22a483358655e.jpg',
 'b10b64a02b75af3cc86a5e19583cd3df.jpg',
 'eeecc8a6eec73b9a591e53cf08e9ce11.jpg',
 'ceab0752ebeb7a2b84dee8a4aca2a04f.jpg',
 '6519cb5f907840d4b908c24c447fb98c.jpg',
 '1d55a369702ac280c4c63847164d82cd.jpg',
 '92eceef4a86ae97c2984a57926bbd6a9.jpg',
 '5b3422fbb36f8345b688b28c8d47ad56.jpg',
 '4d93878e8ffeae861b1f8277e5beedf5.jpg',
 '6f129a34dcddab0317a441e807a3b462.jpg',
 '6802bd8490d70a982a6ecdbc7041eb37.jpg',
 '1d57562374fc0737701219866af45507.jpg',
 'ff6422e43b4a232ae10da5661453bb9e.jpg',
 '5cdeaa339b4ca1052a882bcdd26d5483.jpg',
 'a45a3c484577cd